In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import Transformer
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math
import os
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from collections import Counter
import unicodedata
from tqdm.auto import tqdm
import random
import json

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

MAX_SEQ_LENGTH = 20
BATCH_SIZE = 512
EMBED_DIM = 512
NUM_HEADS = 16
NUM_ENCODER_LAYERS = 8
NUM_DECODER_LAYERS = 8
FFN_DIM = 2048
DROPOUT = 0.1
NUM_EPOCHS = 50
LEARNING_RATE = 0.0001
ERROR_TYPE_WEIGHTS = {
    'DELETION': 2.2,
    'INSERTION': 5.1,
    'KEYBOARD': 12.7,
    'TRANSLITERATION': 1.0
}
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TARGET_LANGUAGE = None

class SpellingDataset(Dataset):
    def __init__(self, data_file, char_to_idx, idx_to_char, max_length=MAX_SEQ_LENGTH, validate_pairs=False):
        print(f"Loading dataset from {data_file}")
        self.data = pd.read_csv(data_file)
        
        if 'LANGUAGE' in self.data.columns:
            languages = self.data['LANGUAGE'].unique()
            print(f"Languages found in dataset: {languages}")
            language_counts = self.data['LANGUAGE'].value_counts()
            print("Language distribution:")
            for lang, count in language_counts.items():
                print(f"  {lang}: {count} examples")
        
        if 'ERROR_TYPE' in self.data.columns:
            error_counts = self.data['ERROR_TYPE'].value_counts()
            print("Error type distribution:")
            for error_type, count in error_counts.items():
                print(f"  {error_type}: {count} examples ({count/len(self.data)*100:.2f}%)")
                
        self.char_to_idx = char_to_idx
        self.idx_to_char = idx_to_char
        self.max_length = max_length
        
        if validate_pairs:
            self._validate_pairs()
    
    def _validate_pairs(self):
        char_changes = []
        suspicious_indices = []
        
        language_stats = {}
        has_language_column = 'LANGUAGE' in self.data.columns
        
        for idx, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Validating data pairs"):
            mistake = str(row['MISTAKE'])
            correct = str(row['CORRECT'])
            
            language = str(row['LANGUAGE']) if has_language_column else 'unknown'
            
            if language not in language_stats:
                language_stats[language] = {
                    'char_changes': [],
                    'suspicious': 0,
                    'total': 0
                }
            
            language_stats[language]['total'] += 1
            
            distance = levenshtein_distance(mistake, correct)
            
            max_len = max(len(mistake), len(correct))
            proportion_changed = distance / max_len if max_len > 0 else 0
            char_changes.append(proportion_changed)
            
            language_stats[language]['char_changes'].append(proportion_changed)
            
            if proportion_changed > 0.8:
                suspicious_indices.append(idx)
                language_stats[language]['suspicious'] += 1
        
        print(f"\nData validation statistics:")
        print(f"Average proportion of changed characters: {np.mean(char_changes):.4f}")
        print(f"Median proportion of changed characters: {np.median(char_changes):.4f}")
        print(f"Found {len(suspicious_indices)} suspicious pairs (>80% different)")
        
        if has_language_column:
            print("\nStatistics by language:")
            for lang, stats in language_stats.items():
                if stats['total'] > 0:
                    avg_change = np.mean(stats['char_changes']) if stats['char_changes'] else 0
                    suspicious_percent = (stats['suspicious'] / stats['total']) * 100
                    print(f"{lang}: {stats['total']} examples, avg change: {avg_change:.4f}, "
                          f"suspicious: {suspicious_percent:.2f}% ({stats['suspicious']}/{stats['total']})")
        
        if suspicious_indices:
            print("\nExamples of suspicious pairs:")
            for i in range(min(5, len(suspicious_indices))):
                idx = suspicious_indices[i]
                mistake = str(self.data.loc[idx, 'MISTAKE'])
                correct = str(self.data.loc[idx, 'CORRECT'])
                language = str(self.data.loc[idx, 'LANGUAGE']) if has_language_column else 'unknown'
                print(f"[{language}] MISTAKE: {mistake}, CORRECT: {correct}, "
                      f"Levenshtein: {levenshtein_distance(mistake, correct)}, "
                      f"Proportion: {levenshtein_distance(mistake, correct) / max(len(mistake), len(correct)):.2f}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        mistake = self.data.iloc[idx]['MISTAKE']
        correct = self.data.iloc[idx]['CORRECT']
        
        mistake = str(mistake)
        correct = str(correct)
        
        mistake_ids = self.tokenize(mistake)
        correct_ids = self.tokenize(correct)
        
        src = torch.tensor([self.char_to_idx['<BOS>']] + mistake_ids + [self.char_to_idx['<EOS>']], dtype=torch.long)
        tgt = torch.tensor([self.char_to_idx['<BOS>']] + correct_ids + [self.char_to_idx['<EOS>']], dtype=torch.long)
        
        item_data = {
            'src': src,
            'tgt': tgt,
            'src_text': mistake,
            'tgt_text': correct,
            'idx': idx
        }
        
        if 'LANGUAGE' in self.data.columns:
            item_data['language'] = self.data.iloc[idx]['LANGUAGE']
            
        if 'ERROR_TYPE' in self.data.columns:
            item_data['error_type'] = self.data.iloc[idx]['ERROR_TYPE']
            
        return item_data
    
    def tokenize(self, text):
        text = unicodedata.normalize('NFC', text)
        return [self.char_to_idx.get(c, self.char_to_idx['<UNK>']) for c in text[:self.max_length]]
    
    def get_sample_pairs(self, n=10):
        indices = random.sample(range(len(self.data)), min(n, len(self.data)))
        samples = []
        for idx in indices:
            sample = {
                'mistake': str(self.data.iloc[idx]['MISTAKE']),
                'correct': str(self.data.iloc[idx]['CORRECT']),
                'idx': idx
            }
            
            if 'LANGUAGE' in self.data.columns:
                sample['language'] = self.data.iloc[idx]['LANGUAGE']
            
            if 'ERROR_TYPE' in self.data.columns:
                sample['error_type'] = self.data.iloc[idx]['ERROR_TYPE']
                
            samples.append(sample)
        return samples

def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def create_vocabulary(data_file, min_freq=5):
    print("Loading data for vocabulary creation...")
    read_csv_params = {}
    if pd.__version__ >= '1.3.0':
        read_csv_params['on_bad_lines'] = 'skip'
    
    df = pd.read_csv(data_file, **read_csv_params)
    
    if 'LANGUAGE' in df.columns:
        languages = df['LANGUAGE'].unique()
        print(f"Building vocabulary for all languages: {languages}")
        language_counts = df['LANGUAGE'].value_counts()
        print("Language distribution for vocabulary:")
        for lang, count in language_counts.items():
            print(f"  {lang}: {count} examples")
    
    all_chars = Counter()
    print("Building vocabulary...")
    for mistake, correct in tqdm(zip(df['MISTAKE'], df['CORRECT']), total=len(df), desc="Processing characters"):
        mistake = str(mistake)
        correct = str(correct)
        
        mistake = unicodedata.normalize('NFC', mistake)
        correct = unicodedata.normalize('NFC', correct)
        
        all_chars.update(mistake)
        all_chars.update(correct)
    
    common_chars = [char for char, count in all_chars.items() if count >= min_freq]
    
    special_tokens = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']
    
    char_to_idx = {char: idx for idx, char in enumerate(special_tokens + common_chars)}
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}
    
    print(f"Vocabulary size: {len(char_to_idx)}")
    print(f"Most common characters: {[c for c, _ in all_chars.most_common(20)]}")
    
    return char_to_idx, idx_to_char

def pad_collate(batch):
    batch.sort(key=lambda x: len(x['src']), reverse=True)
    
    src_lengths = [len(example['src']) for example in batch]
    tgt_lengths = [len(example['tgt']) for example in batch]
    
    max_src_len = max(src_lengths)
    max_tgt_len = max(tgt_lengths)
    
    padded_src = torch.zeros(len(batch), max_src_len, dtype=torch.long)
    padded_tgt = torch.zeros(len(batch), max_tgt_len, dtype=torch.long)
    
    for i, example in enumerate(batch):
        src_len = src_lengths[i]
        tgt_len = tgt_lengths[i]
        
        padded_src[i, :src_len] = example['src']
        padded_tgt[i, :tgt_len] = example['tgt']
    
    src_padding_mask = (padded_src == 0)
    tgt_padding_mask = (padded_tgt == 0)
    
    batch_data = {
        'src': padded_src,
        'tgt': padded_tgt,
        'src_mask': generate_square_subsequent_mask(max_src_len),
        'tgt_mask': generate_square_subsequent_mask(max_tgt_len),
        'src_padding_mask': src_padding_mask,
        'tgt_padding_mask': tgt_padding_mask,
        'src_lengths': src_lengths,
        'tgt_lengths': tgt_lengths,
        'src_text': [example['src_text'] for example in batch],
        'tgt_text': [example['tgt_text'] for example in batch],
        'idx': [example['idx'] for example in batch],
        'language': [example.get('language', 'unknown') for example in batch]
    }
    
    if 'error_type' in batch[0]:
        batch_data['error_type'] = [example.get('error_type', 'unknown') for example in batch]
    
    return batch_data

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.to(DEVICE)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class SpellingTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=EMBED_DIM, nhead=NUM_HEADS, 
                 num_encoder_layers=NUM_ENCODER_LAYERS, num_decoder_layers=NUM_DECODER_LAYERS, 
                 dim_feedforward=FFN_DIM, dropout=DROPOUT):
        super(SpellingTransformer, self).__init__()
        
        self.d_model = d_model
        
        self.src_embedding = nn.Embedding(vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout)
        
        self.transformer = Transformer(
            d_model=d_model, 
            nhead=nhead, 
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False
        )
        
        self.output_layer = nn.Linear(d_model, vocab_size)
        
        self._init_parameters()
    
    def _init_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_padding_mask=None, tgt_padding_mask=None):
        src = src.transpose(0, 1)
        tgt = tgt.transpose(0, 1)
        
        src = self.positional_encoding(self.src_embedding(src) * math.sqrt(self.d_model))
        tgt = self.positional_encoding(self.tgt_embedding(tgt) * math.sqrt(self.d_model))
        
        output = self.transformer(
            src=src,
            tgt=tgt,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask
        )
        
        output = self.output_layer(output)
        
        return output.transpose(0, 1)

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, char_to_idx, weights_dict=ERROR_TYPE_WEIGHTS, ignore_index=-100):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.ignore_index = ignore_index
        self.weights_dict = weights_dict
        self.base_criterion = nn.CrossEntropyLoss(ignore_index=ignore_index, reduction='none')
        self.char_to_idx = char_to_idx
        
    def forward(self, output_flat, target_flat, error_types=None):
        per_token_loss = self.base_criterion(output_flat, target_flat)
        
        if error_types is None:
            return per_token_loss.mean()
        
        seq_len = per_token_loss.size(0) // len(error_types)
        
        weights = torch.ones_like(per_token_loss)
        
        for i, error_type in enumerate(error_types):
            sample_weight = self.weights_dict.get(error_type, 1.0)
            
            start_idx = i * seq_len
            end_idx = (i + 1) * seq_len
            weights[start_idx:end_idx] = sample_weight
            
        weighted_loss = (per_token_loss * weights).sum() / weights.sum()
        
        return weighted_loss

def split_data(input_file, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1"
    
    print(f"Loading dataset from {input_file}...")
    read_csv_params = {}
    if pd.__version__ >= '1.3.0':
        read_csv_params['on_bad_lines'] = 'skip'
    
    df = pd.read_csv(input_file, **read_csv_params)
    
    if 'LANGUAGE' in df.columns:
        languages = df['LANGUAGE'].unique()
        print(f"Using all available languages: {languages}")
        language_counts = df['LANGUAGE'].value_counts()
        print("Language distribution in the dataset:")
        for lang, count in language_counts.items():
            print(f"  {lang}: {count} examples ({count/len(df)*100:.2f}%)")
    
    print(f"Total dataset size: {len(df)}")
    if 'ERROR_TYPE' in df.columns:
        print("Error type distribution:")
        print(df['ERROR_TYPE'].value_counts().head(10))
    
    print("Checking some sample pairs:")
    for i in range(5):
        idx = random.randint(0, len(df) - 1)
        mistake = str(df.iloc[idx]['MISTAKE'])
        correct = str(df.iloc[idx]['CORRECT'])
        language = str(df.iloc[idx]['LANGUAGE']) if 'LANGUAGE' in df.columns else 'unknown'
        error_type = str(df.iloc[idx]['ERROR_TYPE']) if 'ERROR_TYPE' in df.columns else 'unknown'
        print(f"Sample {i+1} [{language}][{error_type}]: MISTAKE={mistake}, CORRECT={correct}")
    
    os.makedirs('data', exist_ok=True)
    
    if 'LANGUAGE' in df.columns and 'ERROR_TYPE' in df.columns:
        print("Performing stratified split by LANGUAGE and ERROR_TYPE...")
        df['strat_column'] = df['LANGUAGE'] + '_' + df['ERROR_TYPE']
        
        train_df, temp_df = train_test_split(
            df, test_size=(val_ratio + test_ratio), 
            random_state=42,
            stratify=df['strat_column']
        )
        
        val_df, test_df = train_test_split(
            temp_df, 
            test_size=test_ratio/(val_ratio + test_ratio), 
            random_state=42,
            stratify=temp_df['strat_column']
        )
        
        train_df = train_df.drop(columns=['strat_column'])
        val_df = val_df.drop(columns=['strat_column'])
        test_df = test_df.drop(columns=['strat_column'])
        
    elif 'ERROR_TYPE' in df.columns:
        print("Performing stratified split by ERROR_TYPE...")
        train_df, temp_df = train_test_split(
            df, test_size=(val_ratio + test_ratio), 
            random_state=42,
            stratify=df['ERROR_TYPE']
        )
        
        val_df, test_df = train_test_split(
            temp_df, 
            test_size=test_ratio/(val_ratio + test_ratio), 
            random_state=42,
            stratify=temp_df['ERROR_TYPE']
        )
    else:
        print("Performing random split...")
        train_df, temp_df = train_test_split(df, test_size=(val_ratio + test_ratio), random_state=42)
        
        val_df, test_df = train_test_split(temp_df, test_size=test_ratio/(val_ratio + test_ratio), random_state=42)
    
    if 'LANGUAGE' in df.columns:
        print("\nLanguage distribution in splits:")
        
        for split_name, split_df in [("Train", train_df), ("Validation", val_df), ("Test", test_df)]:
            print(f"\n{split_name} set language distribution:")
            lang_counts = split_df['LANGUAGE'].value_counts()
            for lang, count in lang_counts.items():
                print(f"  {lang}: {count} examples ({count/len(split_df)*100:.2f}%)")
    
    if 'ERROR_TYPE' in df.columns:
        print("\nError type distribution in splits:")
        
        for split_name, split_df in [("Train", train_df), ("Validation", val_df), ("Test", test_df)]:
            print(f"\n{split_name} set error type distribution:")
            error_counts = split_df['ERROR_TYPE'].value_counts()
            for error_type, count in error_counts.items():
                print(f"  {error_type}: {count} examples ({count/len(split_df)*100:.2f}%)")
    
    train_df.to_csv('data/train.csv', index=False)
    val_df.to_csv('data/val.csv', index=False)
    test_df.to_csv('data/test.csv', index=False)
    
    print(f"Data split complete. Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")
    
    return 'data/train.csv', 'data/val.csv', 'data/test.csv'

def debug_data_loading(data_loader, num_batches=2):
    print("\nDebugging DataLoader...")
    for i, batch in enumerate(data_loader):
        if i >= num_batches:
            break
        
        print(f"\nBatch {i+1} info:")
        print(f"Batch size: {len(batch['src_text'])}")
        print(f"Source shapes: {batch['src'].shape}")
        print(f"Target shapes: {batch['tgt'].shape}")
        
        for j in range(min(3, len(batch['src_text']))):
            print(f"Sample {j}:")
            print(f"  Source text: {batch['src_text'][j]}")
            print(f"  Target text: {batch['tgt_text'][j]}")
            print(f"  Language: {batch['language'][j]}")
            
            if 'error_type' in batch:
                print(f"  Error type: {batch['error_type'][j]}")
                
            print(f"  Index: {batch['idx'][j]}")

def train_model(model, train_loader, val_loader, test_loader, char_to_idx, idx_to_char, 
                num_epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE, save_dir='checkpoints_model'):
    os.makedirs(save_dir, exist_ok=True)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='max',
        factor=0.5,
        patience=2,
        verbose=True,
        min_lr=1e-6
    )
    
    has_error_types = 'error_type' in next(iter(train_loader))
    if has_error_types:
        print(f"Using weighted loss with weights: {ERROR_TYPE_WEIGHTS}")
        criterion = WeightedCrossEntropyLoss(char_to_idx, ignore_index=char_to_idx['<PAD>'])
    else:
        print("Using standard CrossEntropyLoss (no error types available)")
        criterion = nn.CrossEntropyLoss(ignore_index=char_to_idx['<PAD>'])
    
    best_f1 = 0.0
    training_history = {
        'train_loss': [],
        'val_loss': [],
        'f1_scores': [],
        'learning_rates': [],
        'error_type_performance': {}
    }
    
    epoch_pbar = tqdm(range(num_epochs), desc="Training", position=0)
    
    for epoch in epoch_pbar:
        current_lr = optimizer.param_groups[0]['lr']
        training_history['learning_rates'].append(current_lr)
        
        model.train()
        train_loss = 0.0
        
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train] LR: {current_lr:.6f}", 
                          leave=False, position=1, total=len(train_loader))
        
        for batch in train_pbar:
            src = batch['src'].to(DEVICE)
            tgt = batch['tgt'].to(DEVICE)
            src_mask = batch['src_mask'].to(DEVICE)
            tgt_mask = batch['tgt_mask'].to(DEVICE)
            src_padding_mask = batch['src_padding_mask'].to(DEVICE)
            tgt_padding_mask = batch['tgt_padding_mask'].to(DEVICE)
            
            optimizer.zero_grad()
            
            output = model(
                src=src,
                tgt=tgt[:, :-1],
                src_mask=src_mask,
                tgt_mask=tgt_mask[:-1, :-1],
                src_padding_mask=src_padding_mask,
                tgt_padding_mask=tgt_padding_mask[:, :-1]
            )
            
            output_flat = output.reshape(-1, output.shape[-1])
            target_flat = tgt[:, 1:].reshape(-1)
            
            if has_error_types:
                error_types = batch['error_type']
                loss = criterion(output_flat, target_flat, error_types)
            else:
                loss = criterion(output_flat, target_flat)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_pbar.set_postfix(loss=f"{loss.item():.4f}")
        
        avg_train_loss = train_loss / len(train_loader)
        training_history['train_loss'].append(avg_train_loss)
        
        model.eval()
        val_loss = 0.0
        
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Valid]", 
                        leave=False, position=1, total=len(val_loader))
        
        with torch.no_grad():
            for batch in val_pbar:
                src = batch['src'].to(DEVICE)
                tgt = batch['tgt'].to(DEVICE)
                src_mask = batch['src_mask'].to(DEVICE)
                tgt_mask = batch['tgt_mask'].to(DEVICE)
                src_padding_mask = batch['src_padding_mask'].to(DEVICE)
                tgt_padding_mask = batch['tgt_padding_mask'].to(DEVICE)
                
                output = model(
                    src=src,
                    tgt=tgt[:, :-1],
                    src_mask=src_mask,
                    tgt_mask=tgt_mask[:-1, :-1],
                    src_padding_mask=src_padding_mask,
                    tgt_padding_mask=tgt_padding_mask[:, :-1]
                )
                
                output_flat = output.reshape(-1, output.shape[-1])
                target_flat = tgt[:, 1:].reshape(-1)
                
                if isinstance(criterion, WeightedCrossEntropyLoss):
                    loss = criterion.base_criterion(output_flat, target_flat).mean()
                else:
                    loss = criterion(output_flat, target_flat)
                
                val_loss += loss.item()
                val_pbar.set_postfix(loss=f"{loss.item():.4f}")
        
        avg_val_loss = val_loss / len(val_loader)
        training_history['val_loss'].append(avg_val_loss)
        
        f1, detailed_metrics = evaluate_model(model, test_loader, char_to_idx, idx_to_char)
        training_history['f1_scores'].append(f1)
        
        if 'error_type_performance' in detailed_metrics:
            for error_type, score in detailed_metrics['error_type_performance'].items():
                if error_type not in training_history['error_type_performance']:
                    training_history['error_type_performance'][error_type] = []
                training_history['error_type_performance'][error_type].append(score)
        
        scheduler.step(f1)
        
        new_lr = optimizer.param_groups[0]['lr']
        if new_lr != current_lr:
            print(f"\nLearning rate adjusted from {current_lr:.6f} to {new_lr:.6f}")
        
        epoch_pbar.set_postfix(train_loss=f"{avg_train_loss:.4f}", 
                              val_loss=f"{avg_val_loss:.4f}", 
                              f1=f"{f1:.4f}",
                              lr=f"{new_lr:.6f}")
        
        if 'error_type_performance' in detailed_metrics and 'DELETION' in detailed_metrics['error_type_performance']:
            deletion_f1 = detailed_metrics['error_type_performance']['DELETION']
            print(f"\nDELETION error performance: {deletion_f1:.4f}")
        
        if f1 > best_f1:
            best_f1 = f1
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'f1_score': f1,
                'detailed_metrics': detailed_metrics,
                'char_to_idx': char_to_idx,
                'idx_to_char': idx_to_char
            }
            torch.save(checkpoint, os.path.join(save_dir, 'best_model.pt'))
            print(f"Saved new best model with F1: {f1:.4f}")
        
        with open(os.path.join(save_dir, 'training_history.json'), 'w') as f:
            json.dump(training_history, f)
        
        last_checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'f1_score': f1,
            'best_f1': best_f1,
            'char_to_idx': char_to_idx,
            'idx_to_char': idx_to_char
        }
        torch.save(last_checkpoint, os.path.join(save_dir, 'last_checkpoint.pt'))
    
    return model, training_history

def predict_batch(model, src, src_mask, src_padding_mask, char_to_idx, idx_to_char, max_len=MAX_SEQ_LENGTH+2):
    model.eval()
    
    batch_size = src.size(0)
    bos_idx = char_to_idx['<BOS>']
    eos_idx = char_to_idx['<EOS>']
    
    decoder_input = torch.ones(batch_size, 1, dtype=torch.long, device=DEVICE) * bos_idx
    
    completed_sequences = [[] for _ in range(batch_size)]
    has_completed = [False] * batch_size
    
    for i in range(max_len - 1):
        tgt_mask = generate_square_subsequent_mask(decoder_input.size(1))
        
        with torch.no_grad():
            output = model(
                src=src,
                tgt=decoder_input,
                src_mask=src_mask,
                tgt_mask=tgt_mask,
                src_padding_mask=src_padding_mask,
                tgt_padding_mask=None
            )
        
        next_token_logits = output[:, -1, :]
        next_token = torch.argmax(next_token_logits, dim=1).unsqueeze(1)
        
        decoder_input = torch.cat([decoder_input, next_token], dim=1)
        
        for j in range(batch_size):
            if not has_completed[j] and next_token[j].item() == eos_idx:
                seq = decoder_input[j, 1:-1].cpu().tolist()
                completed_sequences[j] = seq
                has_completed[j] = True
        
        if all(has_completed):
            break
    
    for j in range(batch_size):
        if not has_completed[j]:
            seq = decoder_input[j, 1:].cpu().tolist()
            if eos_idx in seq:
                seq = seq[:seq.index(eos_idx)]
            completed_sequences[j] = seq
    
    predicted_words = [''.join(idx_to_char[idx] for idx in seq) for seq in completed_sequences]
    
    return predicted_words

def fine_tune_on_deletion_errors(model, char_to_idx, idx_to_char, 
                         learning_rate=LEARNING_RATE/5, num_epochs=10, save_dir='checkpoints_model_deletion'):
    os.makedirs(save_dir, exist_ok=True)
    print("\n--- Starting fine-tuning specifically for DELETION errors ---")
    
    print("Creating dataset with only DELETION errors...")
    
    train_file = 'data/train.csv'
    val_file = 'data/val.csv'
    test_file = 'data/test.csv'
    
    train_df = pd.read_csv(train_file)
    train_deletion_df = train_df[train_df['ERROR_TYPE'] == 'DELETION']
    
    if len(train_deletion_df) < 100:
        print(f"Found only {len(train_deletion_df)} DELETION examples. Adding more data...")
        other_df = train_df[train_df['ERROR_TYPE'] != 'DELETION'].sample(
            min(len(train_df) // 4, 5000),
            random_state=42
        )
        train_deletion_df = pd.concat([train_deletion_df, other_df])
    
    deletion_train_file = 'data/train_deletion.csv'
    train_deletion_df.to_csv(deletion_train_file, index=False)
    
    print(f"Created fine-tuning dataset with {len(train_deletion_df)} examples")
    print(f"DELETION examples: {len(train_deletion_df[train_deletion_df['ERROR_TYPE'] == 'DELETION'])}")
    
    train_deletion_dataset = SpellingDataset(deletion_train_file, char_to_idx, idx_to_char)
    val_dataset = SpellingDataset(val_file, char_to_idx, idx_to_char)
    test_dataset = SpellingDataset(test_file, char_to_idx, idx_to_char)
    
    train_loader = DataLoader(train_deletion_dataset, batch_size=BATCH_SIZE//2, shuffle=True, collate_fn=pad_collate)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_collate)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_collate)
    
    deletion_focused_weights = ERROR_TYPE_WEIGHTS.copy()
    deletion_focused_weights['DELETION'] = 5.0
    
    model, history = train_model(
        model, 
        train_loader, 
        val_loader, 
        test_loader, 
        char_to_idx, 
        idx_to_char,
        num_epochs=num_epochs,
        learning_rate=learning_rate,
        save_dir=save_dir
    )
    
    return model, history

def evaluate_model(model, data_loader, char_to_idx, idx_to_char):
    model.eval()
    all_preds = []
    all_targets = []
    all_sources = []
    all_indices = []
    all_languages = []
    all_error_types = []
    
    test_pbar = tqdm(data_loader, desc="Evaluating", leave=False, position=1)
    
    correct_char_count = 0
    total_char_count = 0
    
    with torch.no_grad():
        for batch in test_pbar:
            src = batch['src'].to(DEVICE)
            tgt = batch['tgt']
            src_mask = batch['src_mask'].to(DEVICE)
            src_padding_mask = batch['src_padding_mask'].to(DEVICE)
            src_text = batch['src_text']
            tgt_text = batch['tgt_text']
            indices = batch['idx']
            languages = batch['language']
            
            predicted_words = predict_batch(model, src, src_mask, src_padding_mask, char_to_idx, idx_to_char)
            
            all_preds.extend(predicted_words)
            all_targets.extend(tgt_text)
            all_sources.extend(src_text)
            all_indices.extend(indices)
            all_languages.extend(languages)
            
            if 'error_type' in batch:
                all_error_types.extend(batch['error_type'])
            
            for pred, target in zip(predicted_words, tgt_text):
                for p_char, t_char in zip(pred, target):
                    if p_char == t_char:
                        correct_char_count += 1
                    total_char_count += 1
    
    correct_word_count = sum(1 for pred, target in zip(all_preds, all_targets) if pred == target)
    total_words = len(all_targets)
    
    precision = correct_word_count / len(all_preds) if all_preds else 0
    recall = correct_word_count / len(all_targets) if all_targets else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    
    char_accuracy = correct_char_count / total_char_count if total_char_count else 0
    
    distances = [levenshtein_distance(pred, target) for pred, target in zip(all_preds, all_targets)]
    avg_distance = sum(distances) / len(distances) if distances else 0
    
    error_type_performance = {}
    
    if all_error_types:
        for i, error_type in enumerate(all_error_types):
            if error_type not in error_type_performance:
                error_type_performance[error_type] = {'correct': 0, 'total': 0}
            
            error_type_performance[error_type]['total'] += 1
            if all_preds[i] == all_targets[i]:
                error_type_performance[error_type]['correct'] += 1
    elif 'ERROR_TYPE' in data_loader.dataset.data.columns:
        for i, idx in enumerate(all_indices):
            error_type = data_loader.dataset.data.iloc[idx]['ERROR_TYPE']
            if error_type not in error_type_performance:
                error_type_performance[error_type] = {'correct': 0, 'total': 0}
            
            error_type_performance[error_type]['total'] += 1
            if all_preds[i] == all_targets[i]:
                error_type_performance[error_type]['correct'] += 1
    
    detailed_metrics = {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'char_accuracy': char_accuracy,
        'avg_levenshtein': avg_distance,
        'error_type_performance': {k: v['correct']/v['total'] if v['total'] > 0 else 0 
                                   for k, v in error_type_performance.items()}
    }
    
    correct_by_language = {}
    incorrect_by_language = {}
    
    for i, (src, tgt, pred, lang) in enumerate(zip(all_sources, all_targets, all_preds, all_languages)):
        if tgt == pred:
            if lang not in correct_by_language:
                correct_by_language[lang] = []
            correct_by_language[lang].append((src, tgt, pred))
        else:
            if lang not in incorrect_by_language:
                incorrect_by_language[lang] = []
            incorrect_by_language[lang].append((src, tgt, pred))
    
    print("\nExamples of CORRECT predictions by language:")
    for language, predictions in correct_by_language.items():
        print(f"\n{language.upper()}:")
        for i in range(min(3, len(predictions))):
            src, tgt, pred = predictions[i]
            print(f"Source: {src}, Target: {tgt}, Predicted: {pred}")
    
    print("\nExamples of INCORRECT predictions by language:")
    for language, predictions in incorrect_by_language.items():
        print(f"\n{language.upper()}:")
        for i in range(min(3, len(predictions))):
            src, tgt, pred = predictions[i]
            print(f"Source: {src}, Target: {tgt}, Predicted: {pred}")
    
    print(f"\nWord-level metrics - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    print(f"Character-level accuracy: {char_accuracy:.4f}")
    print(f"Average Levenshtein distance: {avg_distance:.2f}")
    
    if error_type_performance:
        print("\nPerformance by error type:")
        for error_type, metrics in sorted(error_type_performance.items(), 
                                         key=lambda x: x[1]['correct']/x[1]['total'] if x[1]['total'] > 0 else 0,
                                         reverse=True):
            accuracy = metrics['correct'] / metrics['total'] if metrics['total'] > 0 else 0
            print(f"{error_type}: {accuracy:.4f} ({metrics['correct']}/{metrics['total']})")
    
    language_performance = {}
    for i, lang in enumerate(all_languages):
        if lang not in language_performance:
            language_performance[lang] = {'correct': 0, 'total': 0}
        
        language_performance[lang]['total'] += 1
        if all_preds[i] == all_targets[i]:
            language_performance[lang]['correct'] += 1
    
    if language_performance:
        print("\nPerformance by language:")
        for language, metrics in sorted(language_performance.items(), 
                                       key=lambda x: x[1]['correct']/x[1]['total'] if x[1]['total'] > 0 else 0,
                                       reverse=True):
            accuracy = metrics['correct'] / metrics['total'] if metrics['total'] > 0 else 0
            print(f"{language}: {accuracy:.4f} ({metrics['correct']}/{metrics['total']})")
            
    detailed_metrics['language_performance'] = {k: v['correct']/v['total'] if v['total'] > 0 else 0 
                                                for k, v in language_performance.items()}
    
    test_pbar.close()
    
    return f1, detailed_metrics

def main():
    input_file = 'merged_sorted_output_with_language_az3.csv'
    train_file, val_file, test_file = split_data(input_file)
    
    char_to_idx, idx_to_char = create_vocabulary(input_file)
    vocab_size = len(char_to_idx)
    print(f"Vocabulary size: {vocab_size}")
    
    train_dataset = SpellingDataset(train_file, char_to_idx, idx_to_char, validate_pairs=True)
    val_dataset = SpellingDataset(val_file, char_to_idx, idx_to_char)
    test_dataset = SpellingDataset(test_file, char_to_idx, idx_to_char)
    
    print("\nSample pairs from training dataset:")
    for sample in train_dataset.get_sample_pairs(5):
        language = sample.get('language', 'unknown')
        error_type = sample.get('error_type', 'unknown')
        print(f"[{language}][{error_type}] MISTAKE: {sample['mistake']}, CORRECT: {sample['correct']}")
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_collate)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_collate)
    
    debug_data_loading(test_loader)
    
    model = SpellingTransformer(vocab_size).to(DEVICE)
    print(f"Model initialized on {DEVICE}")
    
    print("\n--- Starting primary training phase with weighted loss ---")
    model, history = train_model(model, train_loader, val_loader, test_loader, char_to_idx, idx_to_char)
    
    checkpoint = torch.load('checkpoints_model/best_model.pt')
    best_model = SpellingTransformer(len(char_to_idx)).to(DEVICE)
    best_model.load_state_dict(checkpoint['model_state_dict'])
    
    fine_tuned_model, fine_tuning_history = fine_tune_on_deletion_errors(
        best_model, 
        char_to_idx, 
        idx_to_char
    )
    
    print("\n--- Performing final evaluation on test set ---")
    
    ft_checkpoint = torch.load('checkpoints_model_deletion/best_model.pt')
    final_model = SpellingTransformer(len(char_to_idx)).to(DEVICE)
    final_model.load_state_dict(ft_checkpoint['model_state_dict'])
    
    test_dataset = SpellingDataset(test_file, char_to_idx, idx_to_char)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_collate)
    
    final_f1, detailed_metrics = evaluate_model(final_model, test_loader, char_to_idx, idx_to_char)
    
    print(f"Training complete. Best model F1 score on test set: {final_f1:.4f}")
    
    if 'error_type_performance' in detailed_metrics and 'DELETION' in detailed_metrics['error_type_performance']:
        deletion_f1 = detailed_metrics['error_type_performance']['DELETION']
        print(f"DELETION error performance: {deletion_f1:.4f}")
    
    with open('checkpoints_model_deletion/final_metrics.json', 'w') as f:
        json.dump(detailed_metrics, f, indent=2)
        
    print("\n--- Comparison with baseline model ---")
    baseline_model = SpellingTransformer(len(char_to_idx)).to(DEVICE)
    baseline_model.load_state_dict(checkpoint['model_state_dict'])
    
    baseline_f1, baseline_metrics = evaluate_model(baseline_model, test_loader, char_to_idx, idx_to_char)
    
    print(f"Baseline model F1 score: {baseline_f1:.4f}")
    print(f"Fine-tuned model F1 score: {final_f1:.4f}")
    
    if 'error_type_performance' in baseline_metrics and 'DELETION' in baseline_metrics['error_type_performance']:
        baseline_deletion = baseline_metrics['error_type_performance']['DELETION']
        final_deletion = detailed_metrics['error_type_performance']['DELETION']
        
        print(f"Baseline DELETION performance: {baseline_deletion:.4f}")
        print(f"Fine-tuned DELETION performance: {final_deletion:.4f}")
        print(f"Improvement: {(final_deletion - baseline_deletion) * 100:.2f}%")

if __name__ == "__main__":
    main()

Loading dataset from merged_sorted_output_with_language_az3.csv...
Total dataset size: 682513
Error type distribution:
ERROR_TYPE
TRANSLITERATION    366603
KEYBOARD           243454
INSERTION           52528
DELETION            19928
Name: count, dtype: int64
Checking some sample pairs:
Sample 1 [unknown][KEYBOARD]: MISTAKE=ılfəcon, CORRECT=əlfəcin
Sample 2 [unknown][KEYBOARD]: MISTAKE=füzəldicisu, CORRECT=düzəldicisi
Sample 3 [unknown][KEYBOARD]: MISTAKE=açılab, CORRECT=açılan
Sample 4 [unknown][TRANSLITERATION]: MISTAKE=misqalitirme, CORRECT=misqalıtirmə
Sample 5 [unknown][KEYBOARD]: MISTAKE=ketagya, CORRECT=kütahya
Performing stratified split by ERROR_TYPE...

Error type distribution in splits:

Train set error type distribution:
  TRANSLITERATION: 293282 examples (53.71%)
  KEYBOARD: 194763 examples (35.67%)
  INSERTION: 42022 examples (7.70%)
  DELETION: 15943 examples (2.92%)

Validation set error type distribution:
  TRANSLITERATION: 36660 examples (53.71%)
  KEYBOARD: 24345 exa

Processing characters:   0%|          | 0/682513 [00:00<?, ?it/s]

Vocabulary size: 42
Most common characters: ['ə', 'l', 'i', 'a', 'r', 'm', 'e', 'n', 'ı', 'k', 's', 't', 'd', 'ü', 'ş', 'u', 'y', 'q', 'o', 'c']
Vocabulary size: 42
Loading dataset from data/train.csv
Распределение по типам ошибок:
  TRANSLITERATION: 293282 примеров (53.71%)
  KEYBOARD: 194763 примеров (35.67%)
  INSERTION: 42022 примеров (7.70%)
  DELETION: 15943 примеров (2.92%)


Validating data pairs:   0%|          | 0/546010 [00:00<?, ?it/s]


Data validation statistics:
Average proportion of changed characters: 0.2057
Median proportion of changed characters: 0.2000
Found 17 suspicious pairs (>80% different)

Examples of suspicious pairs:
[unknown] MISTAKE: cixisi, CORRECT: çıxışı, Levenshtein: 5, Proportion: 0.83
[unknown] MISTAKE: o, CORRECT: i, Levenshtein: 1, Proportion: 1.00
[unknown] MISTAKE: isigi, CORRECT: ışığı, Levenshtein: 5, Proportion: 1.00
[unknown] MISTAKE: y, CORRECT: u, Levenshtein: 1, Proportion: 1.00
[unknown] MISTAKE: sus, CORRECT: şüş, Levenshtein: 3, Proportion: 1.00
Loading dataset from data/val.csv
Распределение по типам ошибок:
  TRANSLITERATION: 36660 примеров (53.71%)
  KEYBOARD: 24345 примеров (35.67%)
  INSERTION: 5253 примеров (7.70%)
  DELETION: 1993 примеров (2.92%)
Loading dataset from data/test.csv
Распределение по типам ошибок:
  TRANSLITERATION: 36661 примеров (53.71%)
  KEYBOARD: 24346 примеров (35.67%)
  INSERTION: 5253 примеров (7.70%)
  DELETION: 1992 примеров (2.92%)

Sample pairs fr

...

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(



Examples of CORRECT predictions by language:

UNKNOWN:
Source: muteşəkkilləşdirilmə, Target: mütəşəkkilləşdirilmə, Predicted: mütəşəkkilləşdirilmə
Source: müvazinetleşdirilmə, Target: müvazinətləşdirilmə, Predicted: müvazinətləşdirilmə
Source: ozünuqiymətləndirmə, Target: özünüqiymətləndirmə, Predicted: özünüqiymətləndirmə

Examples of INCORRECT predictions by language:

UNKNOWN:
Source: mutesəkkilleşdirilmək, Target: mütəşəkkilləşdirilmək, Predicted: mütəşəkkilləşdirilmə
Source: dösekcəsi, Target: döşəkçəsi, Predicted: döşəkcəsi
Source: qıfıll, Target: qıfıl, Predicted: qıfılı

Word-level metrics - Precision: 0.9666, Recall: 0.9666, F1: 0.9666
Character-level accuracy: 0.9933
Average Levenshtein distance: 0.06

Performance by error type:
KEYBOARD: 0.9870 (24029/24346)
TRANSLITERATION: 0.9688 (35517/36661)
INSERTION: 0.9282 (4876/5253)
DELETION: 0.7776 (1549/1992)

Performance by language:
unknown: 0.9666 (65971/68252)
Training complete. Best model F1 score on test set: 0.9666
DELETIO

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Evaluating:   0%|          | 0/134 [00:00<?, ?it/s]


Examples of CORRECT predictions by language:

UNKNOWN:
Source: muteşəkkilləşdirilmə, Target: mütəşəkkilləşdirilmə, Predicted: mütəşəkkilləşdirilmə
Source: müvazinetleşdirilmə, Target: müvazinətləşdirilmə, Predicted: müvazinətləşdirilmə
Source: ozünuqiymətləndirmə, Target: özünüqiymətləndirmə, Predicted: özünüqiymətləndirmə

Examples of INCORRECT predictions by language:

UNKNOWN:
Source: mutesəkkilleşdirilmək, Target: mütəşəkkilləşdirilmək, Predicted: mütəşəkkilləşdirilmə
Source: cilçıragi, Target: çilçırağı, Predicted: çılçırağı
Source: dəbəri, Target: səbəti, Predicted: fənəri

Word-level metrics - Precision: 0.9689, Recall: 0.9689, F1: 0.9689
Character-level accuracy: 0.9939
Average Levenshtein distance: 0.05

Performance by error type:
KEYBOARD: 0.9868 (24025/24346)
TRANSLITERATION: 0.9728 (35664/36661)
INSERTION: 0.9395 (4935/5253)
DELETION: 0.7545 (1503/1992)

Performance by language:
unknown: 0.9689 (66127/68252)
Baseline model F1 score: 0.9689
Fine-tuned model F1 score: 0.9666